**First Step**
Import Libraries


In [8]:
import requests
from datetime import datetime, timedelta
from config import API_KEY

# API URLs
market_status_url = f'https://api.polygon.io/v1/marketstatus/now?apiKey={API_KEY}'
market_holiday_url = f'https://api.polygon.io/v1/marketstatus/upcoming?apiKey={API_KEY}'

def market_open_closed():
    response = requests.get(market_status_url)
    data = response.json()

    market_closed = data['market'] == 'closed'
    market_close_reason = ""

    if market_closed:
        today = datetime.now()
        if today.weekday() in [5, 6]: 
            market_close_reason = "Weekend"
        else:
            holidays = market_holiday() 
            holiday_names = [holiday['name'] for holiday in holidays if holiday['status'] == 'closed']
            if holiday_names:
                market_close_reason = f"The market is closed due to holiday(s): {', '.join(holiday_names)}."
          

    filtered_data = {
        'afterHours': data.get('afterHours', 'N/A'),
        'market': data.get('market', 'N/A'),
        's_and_p': data['indicesGroups'].get('s_and_p', 'N/A'),
        'nasdaq': data['indicesGroups'].get('nasdaq', 'N/A'),
        'dow_jones': data['indicesGroups'].get('dow_jones', 'N/A'),
        'ftse_russell': data['indicesGroups'].get('ftse_russell', 'N/A'),
        'closure_reason': market_close_reason
    }
    
    return filtered_data

def market_holiday():
    response = requests.get(market_holiday_url)
    data = response.json()
    return data

def next_holiday():
    holidays = market_holiday()
    today = datetime.now()
    next_month_date = today + timedelta(days=30)  
    upcoming = []

    for holiday in holidays:
        holiday_date = datetime.strptime(holiday['date'], '%Y-%m-%d')  
        if holiday['status'] == 'closed' and today <= holiday_date <= next_month_date:
            upcoming.append(holiday)

    return upcoming

def get_market_status():
    market_status = market_open_closed()
    upcoming_holidays = next_holiday()
    return market_status, upcoming_holidays


market_status, next_holidays = get_market_status()
print("Market Status:", market_status)
if next_holidays:
    print("Upcoming Market Holidays in the next 30 days :")
    for holiday in next_holidays:
        print(f"{holiday['name']} on {holiday['date']} (Exchange: {holiday['exchange']})")
else:
    print("No upcoming holidays within the next month.")




Market Status: {'afterHours': False, 'market': 'closed', 's_and_p': 'closed', 'nasdaq': 'closed', 'dow_jones': 'closed', 'ftse_russell': 'closed', 'closure_reason': 'Weekend'}
Upcoming Market Holidays in the next 30 days :
Thanksgiving on 2024-11-28 (Exchange: NASDAQ)
Thanksgiving on 2024-11-28 (Exchange: NYSE)
